In [32]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\holla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'gensim'

In [ ]:
!conda install -c anaconda gensim

In [2]:
df = pd.read_csv('assignment_data.csv')

In [3]:
df.head()

,SetID,Adverse Reactions,Summary
0,a834d1cf-72fc-93bf-e053-2995a90a6191,The following adverse events were observed and...,NaN
1,a835b697-2beb-1ba8-e053-2995a90a470c,The following serious adverse reactions are de...,NaN
2,a837f13e-fafc-0535-e053-2995a90a5070,ADVERSE REACTIONS Clinical Trials Experience I...,NaN
3,a838204b-9564-9aa6-e053-2a95a90af02f,ADVERSE REACTIONS Clinical Trials Experience I...,NaN
4,f265e6dd-f47e-4511-9468-282184bcd1b1,The most common adverse reactions leading to d...,NaN


In [4]:
len(df)

990

In [23]:
df = df[df['Adverse Reactions'].notnull()]
df = df[df['Adverse Reactions'] != "ERROR1"]
df = df[df['Adverse Reactions'] != "Adverse Reactions"]

In [24]:
len(df)

989

In [26]:
sentences = []

for sentence in df['Adverse Reactions']:
    sentences.append(sent_tokenize(sentence))
    

In [30]:
df['sentences'] = sentences

In [31]:
df.head()

,SetID,Adverse Reactions,Summary,sentences
0,a834d1cf-72fc-93bf-e053-2995a90a6191,The following adverse events were observed and...,NaN,[The following adverse events were observed an...
1,a835b697-2beb-1ba8-e053-2995a90a470c,The following serious adverse reactions are de...,NaN,[The following serious adverse reactions are d...
2,a837f13e-fafc-0535-e053-2995a90a5070,ADVERSE REACTIONS Clinical Trials Experience I...,NaN,[ADVERSE REACTIONS Clinical Trials Experience ...
3,a838204b-9564-9aa6-e053-2a95a90af02f,ADVERSE REACTIONS Clinical Trials Experience I...,NaN,[ADVERSE REACTIONS Clinical Trials Experience ...
4,f265e6dd-f47e-4511-9468-282184bcd1b1,The most common adverse reactions leading to d...,NaN,[The most common adverse reactions leading to ...
